1.Importando as Bibliotecas

In [1]:
import pandas as pd
from joblib import load, dump 

2.Carregar Dataframes 

In [2]:
categorias = load("../../util/data/df_categorias.z")
produtos = load("../../util/data/df_produtos.z")
unidades = load("../../util/data/df_unidades.z")
vendas = load("../../util/data/df_vendas.z")

3.Função para mesclar entradas com o nome mais curto e evitar duplicatas

In [3]:
def merge_similar_descriptions(desc1, desc2):
    # Verifica se uma descrição está contida na outra
    if desc1.lower() in desc2.lower() or desc2.lower() in desc1.lower():
        return min(desc1, desc2, key=len)  # Retorna o nome com menos caracteres
    return None 

descs = categorias['descricao'].tolist()
substituicoes = {}

# Comparando todas as descrições
for i, desc1 in enumerate(descs):
    for j, desc2 in enumerate(descs):
        if i != j and desc2 not in substituicoes:
            nova = merge_similar_descriptions(desc1, desc2)
            if nova:
                substituicoes[desc2] = nova


4.Tratammento de Dados - Categorias 

In [4]:
print(categorias['descricao'].dtype)

categorias['descricao'] = categorias['descricao'].astype(str).str.upper().str.replace(r'[^A-Z ]', '', regex=True)
categorias['descricao'] = categorias['descricao'].replace(substituicoes)
categorias.drop_duplicates(subset='descricao', keep='first',inplace= True)

categorias.reset_index(drop=True, inplace=True) 

dump(categorias, "../../util/data/df_categorias.z")

print(categorias)

object
     id                         descricao
0    41                        COMPUTADOR
1    64                             PASTA
2   342                            CANETA
3    51                          PRESENTE
4   346         MATERIAL DE USO E CONSUMO
5    62                          ALIMENTO
6    40                              CABO
7   103            MIDIA DE ARMAZENAMENTO
8   215                             DRONE
9     5                    SOM AUTOMOTIVO
10    8                               GPS
11   10                         TELEVISAO
12   11                            BEBIDA
13    6                         ACESSORIO
14    7                             TINTA
15   14             MATERIAL DE EMBALAGEM
16    3               MATERIAL DE LIMPEZA
17    9                          ENVELOPE
18    2                             LIVRO
19    4                      CAIXA DE SOM
20  365                            MOVEIS
21  366                             SOFAS
22  367                    

.Verifica o tipo de dado da coluna descricao 

.Remove símbolos (mantendo só letras maiúsculas e espaço)


.Elimina as entradas duplicadas e entradas com nomes semelhantes, mantendo apenas a primeira.


5.Tratamento de Dados - Produtos

In [5]:
# produtos com preco_venda ausente ou igual a zero

precos_originais = produtos['preco_venda'].copy()

# Criar uma máscara que marca os valores que serão modificados
mask_modificados = (produtos['preco_venda'].isna()) | (produtos['preco_venda'] == 0) 

medianas = produtos[produtos['preco_venda'] > 0].groupby('categoria')['preco_venda'].median()

for categoria, mediana in medianas.items():
    mask = (produtos['categoria'] == categoria) & mask_modificados
    produtos.loc[mask, 'preco_venda'] = mediana 

produtos_modificados = produtos[mask_modificados]

nulos_preco_venda = produtos['preco_venda'].isnull().sum()

print(f'Número de valores nulos na coluna preco_venda: {nulos_preco_venda}') 


Número de valores nulos na coluna preco_venda: 0


In [6]:
# conversão
produtos['codigo_categoria'] = produtos['codigo_categoria'].astype('Int64')

#Remoção
produtos.drop(columns=['classificacao'], inplace=True, errors='ignore')

# Substituir valores específicos
produtos.loc[produtos['unidade'] == 'UNIDADE', 'unidade'] = 'UN'
produtos.loc[produtos['unidade'] == 'CAIXA', 'unidade'] = 'CX'

# Preencher valores nulos com 'UN'
produtos['unidade'] = produtos['unidade'].apply(lambda x: 'UN' if pd.isna(x) else x) 

# Removendo a coluna 

produtos.drop(columns=['codigo_barras'], inplace=True, errors='ignore')

dump(produtos, "../../util/data/df_produtos.z")

print(produtos)

          id                          descricao unidade  marca  \
1     194412                   CANETA REF 35633      UN    BIC   
3     006467                    CANETA REF 5968      UN    BIC   
4     004559                    CANETA REF 4092      UN    BIC   
5     006654                  ENVELOPE REF 6152      UN     3M   
6     005543                    CANETA REF 5066      UN    BIC   
...      ...                                ...     ...    ...   
9474  188898                   CANETA REF 30243      UN    BIC   
9475  196155  SOFA MAGIC HOLE A4 BRANCO/LARANJA      UN   SONY   
9476  196156         SOFA SLIM LINE 4 LUG CHALK      UN   SONY   
9477  196157         CHAISE NORONHA C/BR LD DIR      UN  GALLO   
9590     NaN           PAPEL DE PAREDE LISTRADO     ROL    NaN   

      codigo_categoria  categoria  preco_venda  
1                  342     CANETA         8.93  
3                  342     CANETA         8.38  
4                  342     CANETA        67.91  
5          

. A coluna 'codigo_categoria' é convertida para tipo Int64, permitindo valores nulos.

. A coluna 'classificacao' é removida, se existir.

. A coluna 'unidade' é padronizada (por exemplo, "UNIDADE" se torna "UN" e "CAIXA" se torna "CX"), assim como os valores nulos
com 'UN'.

. Registros com valor nulo na coluna 'preco_venda' são modificados pela mediana da sua respectiva categoria.

. O DataFrame tratado foi salvo novamente no mesmo arquivo df_produtos.z.

.  Removendo a coluna código de barras.

6.Tratamento de Dados -  Unidades 

In [7]:
cols = ['nome', 'cidade', 'estado']
for col in cols:
    if col in unidades.columns:
        unidades[col] = (unidades[col].astype(str).str.strip().str.upper().str.replace(r'[^A-Z \-]', '', regex=True))
    unidades.drop_duplicates(inplace=True)
    unidades.reset_index(drop=True, inplace=True)
    
    dump(unidades, "../../util/data/df_unidades.z")
    print("\nUnidades tratadas:")
    print(unidades)


Unidades tratadas:
      id    nome_fantasia status
0  10003   L03 - SALVADOR    sim
1  10002     L02 - SOBRAL    sim
2  10001  L01 - FORTALEZA    sim

Unidades tratadas:
      id    nome_fantasia status
0  10003   L03 - SALVADOR    sim
1  10002     L02 - SOBRAL    sim
2  10001  L01 - FORTALEZA    sim

Unidades tratadas:
      id    nome_fantasia status
0  10003   L03 - SALVADOR    sim
1  10002     L02 - SOBRAL    sim
2  10001  L01 - FORTALEZA    sim


Se o DataFrame unidades não estiver vazio:

.Letras maiúsculas, o hífen e o espaço sem outros símbolos ou números nas colunas 'nome', 'cidade' e 'estado', se estas existirem.

.Linhas duplicadas são removidas.

.Os índices são reorganizados após a remoção das duplicatas.

.O DataFrame tratado foi salvo novamente no mesmo arquivo df_unidades.z.

7.Tratamento de Dados - Vendas

In [8]:
if not vendas.empty:
    cols_to_drop = vendas.columns[vendas.isna().mean() > 0.8]
    vendas.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    for col in ['quantidade', 'valor_total']:
        if col in vendas.columns:
            vendas[col] = vendas[col].fillna(0)
    vendas.drop_duplicates(inplace=True)
    vendas.reset_index(drop=True, inplace=True)
    dump(vendas, "../../util/data/df_vendas.z")
    print("\nVendas tratadas:")
    print(vendas)


Vendas tratadas:
       codigo_empresa tipo_pedido  centro_resultado  documento  \
0                   2          PV                 2      24560   
1                   1          PV                 2      22433   
2                   2          PV                 2      24560   
3                   2          PV                 2      24560   
4                   2          PV                 2      24560   
...               ...         ...               ...        ...   
10413               3          OA                 7      24593   
10414               3          OA                 7      24593   
10415               3          OA                 7      24593   
10416               3          OA                 7      24593   
10417               1          PV                 7      24601   

       codigo_cliente  codigo_empregado data_emissao hora_emissao  \
0                4395                56   2023-03-21     12:10:07   
1                 236                 9   2023-02-1

Se o DataFrame vendas não estiver vazio:

.Colunas com mais de 80% de valores ausentes (NaN) são removidas.

.As colunas 'quantidade' e 'valor_total' têm os valores nulos preenchidos com zero.

.Linhas duplicadas são removidas.

.Os índices são reorganizados após a remoção das duplicatas.

.O DataFrame tratado foi salvo novamente no mesmo arquivo df_vendas.z.